# Debugging notes:

#### Multiple Config files

When you are looking at simulations inside SimulationsMain/, there may be multiple Config files that load in different packages for one simulation at different levels of the directory. For example, if we look at the example ``SimulationMain/magnetoHD/Torus/`` which simulates an accretion torus (donut) with a point mass (black hole or young star) at its center, when you compile it looks at all the ``Config``s in the other modules:

- ``SimulationMain/magnetoHD/Torus/Config`` : loads in gravitational pointmass potential to be place at the center of the Torus
- ``SimulationMain/magnetoHD/Config``: loads in 8 wave solver for MHD
- ``SimulationMain/Config``: loads in basic Hydro stuff


#### Quick fixes to simulation files

When making small changes to the F90 files, rather than changing the F90 files inside ``SimulationMain/<Simulation-module>/``, it is often faster to change the copy of the F90 source code inside ``object/``, then ``make`` compile the source inside object/. If only the object/flash.par is changed, then no compilation is necessary (which is useful when restarting a run). This also seems to automatically copy the updated F90 files to ``SimulationMain/<Simulation-module>/``, but note that the flash.par will not be updated.

# Small densities 

For some hydro solvers, the minimum pressure, energy, density, or any scalar value stored in the simulation can not be smaller than the default 1E-10. This means that if these values every drop below 1E-10, the values would just be coerced to raise up to 1E-10. This can be problematic for astrophysical simulations, where the densities that we are dealing with (even in a "dense" core) is on the order of 1E-19. So we need to override these default values with a lower density cutoff inside ``Config``: 
~~~fortran
D   tiny            Threshold value used for numerical zero
PARAMETER tiny         REAL     1.e-30

D  smallt
PARAMETER smallt      REAL 1.e-30

D  smallp
PARAMETER smallp      REAL 1.e-30

D  smalle
PARAMETER smalle      REAL 1.e-30

D  smlrho
PARAMETER smlrho      REAL 1.e-30
~~~

Then when you do setup, you should see something like this: 
```
    INFO: Parameter smallp defined in both
    physics/Hydro/HydroMain/unsplit (default 1.E-10) and
    Simulation/SimulationMain/RotatingSinkSphere (default 1.e-30)
    Simulation instance overrides; removing other instance.
```

# Parallel Runs



``Too many blocks!  Increase MAXBLOCKS or use more processors.``

For example, I have 24 processors, then I could assign 2 processor in the x direction, 3 processors in the y direction, 4 processors in the z direction (Check that 2x3x4=24proc)
```
    iProcs = 2
    jProcs = 3
    kProcs = 4
```

As a rule of thumb, if your simulation is symmetric, there is the same amount of work to be done in each direction, so it's best to keep the number of processors in each direction equal, so that the code has an easier time balancing the work load.

## Common Errors: 

- Generally, "grepping" around the ``object/`` directory can often be helpful to understand what part of the source code is actually loaded into your simulation and causing the error.

    ``grep -rnw 'object/' -e '<matching-strings>' ``
    
``Terminating execution. DRIVER_ABORT: Nonconvergence in subroutine rieman``
- check the values of the outputs, usually this means something has gone really wrong with the simulation or there is very large velocities in one of the cells.
    
``DRIVER_ABORT: ERROR: cannot locate real runtime parameter.``
- To define a new runtime parameter, you need to specify the variable in Config (declare the variable to the rest of the simulation), Simulation_init (declare/initialize the variable in the Fortran sense),flash.par(give the variable a value)Simulation_data(read the value of the variable), and Simulation_initBlock (use the variable)
- You are probably getting the error because you forgot one of the parts, the ``RuntimeParameters_read:  ignoring unknown parameter "<variable-name>`` should give a good clue to where to start looking